In [542]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from econml.dr import DRLearner
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import (
    StratifiedShuffleSplit,
    GridSearchCV,
    train_test_split,
    cross_validate,
    cross_val_score,
)
from lightgbm import LGBMClassifier
from sklift.models import SoloModel
from sklift.viz import plot_qini_curve
from sklift.datasets import fetch_megafon
from sklift.metrics import make_uplift_scorer
import os
import sys
from pathlib import Path
import yaml
from datetime import datetime
import re


# sys fix

In [543]:
cwd = Path.cwd()
repo_root = ([cwd] + list(cwd.parents))[1]

# Ensure repo_root is on sys.path so `src.train` can be imported
sys.path.append(str(repo_root))
from src.process_datasets import create_data, get_web_feats

# Load the YAML config file
with open(os.path.join(repo_root, "config.yaml"), "r") as f:
    config = yaml.safe_load(f)

# data vis

## web visits


In [544]:
# Load and show web_visits from data/train
web_visits_path = repo_root / "data" / "train" / "web_visits.csv"

web_visits = pd.read_csv(web_visits_path, low_memory=False)

# try to parse a common timestamp column if present
if "timestamp" in web_visits.columns:
    web_visits["timestamp"] = pd.to_datetime(
        web_visits["timestamp"], errors="coerce", dayfirst=True
    )

print("shape:", web_visits.shape)
print("columns:", list(web_visits.columns))
web_visits.head(10)

shape: (259340, 5)
columns: ['member_id', 'url', 'title', 'description', 'timestamp']


,member_id,url,title,description,timestamp
0,1,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:00
1,1,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:00
2,1,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:00
3,2,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:00
4,2,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:00
5,2,https://care.portal/sleep/928,Restorative sleep tips,Sleep apnea screening and hygiene,2025-07-02 13:41:00
6,2,https://guide.wellness/wellness/814,Healthy eating guide,Tips on nutrition and balanced diets,2025-07-01 00:37:00
7,2,https://care.portal/aerobic/384,Aerobic exercise,Cardiovascular fitness and endurance,2025-07-07 14:04:00
8,2,https://guide.wellness/heart/881,Hypertension basics,Blood pressure and lifestyle changes,2025-07-04 16:29:00
9,2,https://health.wellco/diabetes/194,HbA1c targets,Improving glycemic control and blood glucose,2025-07-01 13:19:00


In [545]:
web_visits["timestamp"].max()
outreach_time = datetime.strptime("2025-15-07", "%Y-%d-%m")
outreach_time - web_visits["timestamp"].max()

Timedelta('0 days 00:01:00')

In [546]:
def check_column_dependency(df, col1, col2):
    """
    Check if knowing one column allows you to determine the other.

    Parameters:
    df (pd.DataFrame): The dataframe to check.
    col1 (str): The first column name.
    col2 (str): The second column name.

    Returns:
    dict: A dictionary with results for both directions.
    """
    # Check if col1 -> col2 is one-to-one
    col1_to_col2 = df.groupby(col1)[col2].nunique().max() == 1

    # Check if col2 -> col1 is one-to-one
    col2_to_col1 = df.groupby(col2)[col1].nunique().max() == 1

    return {f"{col1} -> {col2}": col1_to_col2, f"{col2} -> {col1}": col2_to_col1}


# Example usage:
result = check_column_dependency(web_visits, "url", "description")
print(result)
web_visits.groupby("url")["description"].nunique().max()

{'url -> description': False, 'description -> url': False}


2

In [547]:
web_visits.groupby("url")["description"].nunique().sort_values(ascending=False).head(20)

url
https://health.wellco/nutrition/576    2
https://living.better/heart/172        2
https://living.better/heart/163        2
https://living.better/heart/165        2
https://living.better/heart/166        2
https://living.better/heart/167        2
https://living.better/heart/168        2
https://living.better/heart/169        2
https://living.better/heart/170        2
https://living.better/heart/174        2
https://living.better/heart/161        2
https://living.better/heart/175        2
https://living.better/heart/177        2
https://living.better/heart/178        2
https://living.better/heart/180        2
https://living.better/heart/181        2
https://living.better/heart/182        2
https://living.better/heart/183        2
https://living.better/heart/162        2
https://living.better/heart/160        2
Name: description, dtype: int64

In [548]:
web_visits2

,member_id,domain_is_wellco
0,1,True
1,1,False
2,1,True
3,2,False
4,2,False
...,...,...
259335,10000,False
259336,10000,False
259337,10000,False
259338,10000,False


In [549]:
web_visits["url_category"] = web_visits["url"].str.split("/").str[3]
web_visits["domain"] = web_visits["url"].str.split("/").str[2]
# Add time window features for the first and second week of observation
web_visits["days_from_outreach"] = (outreach_time - web_visits["timestamp"]).dt.days
web_visits["weeks_from_outreach"] = (web_visits["days_from_outreach"]+1) // 7 + 1
total = web_visits.groupby("member_id").size().rename("total_web_visits")

web_visits["domain_is_wellco"] = web_visits["domain"].str.contains("wellco")
# Count visits per user in the first and second week
weekly_visits = (
    web_visits.groupby(["member_id", "weeks_from_outreach"])
    .size()
    .unstack(fill_value=0)
)
weekly_visits
weekly_visits.columns = [
    f"weeks_from_outreach_{col}_visits" for col in weekly_visits.columns
]
# # # Count how many domains are WellCo vs not
domain_counts = (
    web_visits.groupby(["member_id", "domain_is_wellco"]).size().unstack(fill_value=0)
)
domain_counts.columns = ["not_wellco_domains", "wellco_domains"]

# # Combine weekly visits and domain counts
aggregated_features = pd.concat(
    [weekly_visits, domain_counts, total], axis=1
).reset_index()

web_visits["category"] = web_visits["url_category"] + "_" + web_visits["title"]
aggregated_features["weekly_visits_diff"] = (
    aggregated_features["weeks_from_outreach_1_visits"]
    - aggregated_features["weeks_from_outreach_2_visits"]
    - aggregated_features["weeks_from_outreach_3_visits"]
)

# # # Merge aggregated features back into the main dataframe
domain_counts = (
    web_visits.groupby(["member_id", "domain_is_wellco"]).size().unstack(fill_value=0)
)
domain_counts["welco_domain_ratio"] = domain_counts[True] / (
    domain_counts[True] + domain_counts[False]
)
aggregated_features = aggregated_features.merge(domain_counts["welco_domain_ratio"], on="member_id", how="left")

web_visits.drop(
    columns=[
        "title",
        "url_category",
        "description",
        "url",
        "timestamp",
        "domain",
        "category",
        "weeks_from_outreach",
        "days_from_outreach",
        "domain_is_wellco",
    ],
    inplace=True,
)
web_visits = aggregated_features

In [550]:
aggregated_features

,member_id,weeks_from_outreach_1_visits,weeks_from_outreach_2_visits,weeks_from_outreach_3_visits,not_wellco_domains,wellco_domains,total_web_visits,weekly_visits_diff,welco_domain_ratio
0,1,1,2,0,1,2,3,-1,0.666667
1,2,1,6,2,7,2,9,-7,0.222222
2,3,23,26,3,46,6,52,-6,0.115385
3,4,5,9,1,14,1,15,-5,0.066667
4,5,3,7,1,11,0,11,-5,0.000000
...,...,...,...,...,...,...,...,...,...
9970,9996,7,8,3,14,4,18,-4,0.222222
9971,9997,3,3,2,6,2,8,-2,0.250000
9972,9998,4,3,1,7,1,8,0,0.125000
9973,9999,15,18,3,33,3,36,-6,0.083333


In [551]:
domain_counts["welco_domain_ratio"]

member_id
1        0.666667
2        0.222222
3        0.115385
4        0.066667
5        0.000000
           ...   
9996     0.222222
9997     0.250000
9998     0.125000
9999     0.083333
10000    0.104167
Name: welco_domain_ratio, Length: 9975, dtype: float64

In [552]:
domain_counts["welco_domain_ratio"] = domain_counts[True] / (
    domain_counts[True] + domain_counts[False])
domain_counts

domain_is_wellco,False,True,welco_domain_ratio
member_id,,,
1,1,2,0.666667
2,7,2,0.222222
3,46,6,0.115385
4,14,1,0.066667
5,11,0,0.000000
...,...,...,...
9996,14,4,0.222222
9997,6,2,0.250000
9998,7,1,0.125000


In [553]:
web_visits

,member_id,weeks_from_outreach_1_visits,weeks_from_outreach_2_visits,weeks_from_outreach_3_visits,not_wellco_domains,wellco_domains,total_web_visits,weekly_visits_diff,welco_domain_ratio
0,1,1,2,0,1,2,3,-1,0.666667
1,2,1,6,2,7,2,9,-7,0.222222
2,3,23,26,3,46,6,52,-6,0.115385
3,4,5,9,1,14,1,15,-5,0.066667
4,5,3,7,1,11,0,11,-5,0.000000
...,...,...,...,...,...,...,...,...,...
9970,9996,7,8,3,14,4,18,-4,0.222222
9971,9997,3,3,2,6,2,8,-2,0.250000
9972,9998,4,3,1,7,1,8,0,0.125000
9973,9999,15,18,3,33,3,36,-6,0.083333


In [554]:
variable_cols = (
    web_visits.groupby("member_id").nunique().apply(lambda col: col > 1).any()
)

cols_to_aggregate_or_remove = variable_cols[variable_cols].index.tolist()
cols_to_aggregate_or_remove

[]

In [555]:
variable_cols

weeks_from_outreach_1_visits    False
weeks_from_outreach_2_visits    False
weeks_from_outreach_3_visits    False
not_wellco_domains              False
wellco_domains                  False
total_web_visits                False
weekly_visits_diff              False
welco_domain_ratio              False
dtype: bool

In [556]:
def process_datasets_web_visits(web_visits, outreach_time):
    web_visits["url_category"] = web_visits["url"].str.split("/").str[3]
    web_visits["domain"] = web_visits["url"].str.split("/").str[2]
    # Add time window features for the first and second week of observation
    web_visits["days_from_outreach"] = (outreach_time - web_visits["timestamp"]).dt.days
    web_visits["weeks_from_outreach"] = (web_visits["days_from_outreach"]+1) // 7 + 1
    total = web_visits.groupby("member_id").size().rename("total_web_visits")

    web_visits["domain_is_wellco"] = web_visits["domain"].str.contains("wellco")
    # Count visits per user in the first and second week
    weekly_visits = (
        web_visits.groupby(["member_id", "weeks_from_outreach"])
        .size()
        .unstack(fill_value=0)
    )
    weekly_visits
    weekly_visits.columns = [
        f"weeks_from_outreach_{col}_visits" for col in weekly_visits.columns
    ]
    # # # Count how many domains are WellCo vs not
    domain_counts = (
        web_visits.groupby(["member_id", "domain_is_wellco"]).size().unstack(fill_value=0)
    )
    domain_counts.columns = ["not_wellco_domains", "wellco_domains"]
    
    # # Combine weekly visits and domain counts
    aggregated_features = pd.concat(
        [weekly_visits, domain_counts, total], axis=1
    ).reset_index()

    web_visits["category"] = web_visits["url_category"] + "_" + web_visits["title"]
    aggregated_features["weekly_visits_diff"] = (
        aggregated_features["weeks_from_outreach_1_visits"]
        - aggregated_features["weeks_from_outreach_2_visits"]
        - aggregated_features["weeks_from_outreach_3_visits"]
    )

    # # # Merge aggregated features back into the main dataframe
    #   # # Merge aggregated features back into the main dataframe
    domain_counts = (
        web_visits.groupby(["member_id", "domain_is_wellco"]).size().unstack(fill_value=0)
    )
    domain_counts["welco_domain_ratio"] = domain_counts[True] / (
        domain_counts[True] + domain_counts[False]
    )
    aggregated_features = aggregated_features.merge(domain_counts["welco_domain_ratio"], on="member_id", how="left")
    web_visits.drop(
        columns=[
            "title",
            "url_category",
            "description",
            "url",
            "timestamp",
            "domain",
            "category",
            "weeks_from_outreach",
            "days_from_outreach",
            "domain_is_wellco",
        ],
        inplace=True,
    )
    web_visits = aggregated_features
    return web_visits

In [557]:
process_datasets_web_visits(web_visits, outreach_time)

KeyError: 'url'

In [ ]:
web_visits_for_merge = process_datasets_web_visits(web_visits, outreach_time)
web_visits_for_merge

KeyError: 'url'

In [ ]:
web_visits_for_merge = web_visits

In [ ]:
# merge aggregated web features (result) into churn labels on member_id and save
churn_path = repo_root / "data" / "train" / "churn_labels.csv"
churn = pd.read_csv(churn_path)

merged = pd.merge(churn, web_visits_for_merge, on="member_id", how="left")

# Fill numeric feature NaNs with 0, non-numeric with empty string
feat_cols = merged.columns.difference(churn.columns)
for c in feat_cols:
    if pd.api.types.is_numeric_dtype(merged[c]):
        merged[c] = merged[c].fillna(0).astype(int)
    else:
        merged[c] = merged[c].fillna("")

out_path = repo_root / "data" / "train" / "churn_labels_with_web_feats.csv"
merged.to_csv(out_path, index=False)
print(f"Saved merged file to {out_path} (shape={merged.shape})")
merged.head()

Saved merged file to c:\Users\elad\Desktop\rnd\VI\wellco_churn\data\train\churn_labels_with_web_feats.csv (shape=(259365, 8))


,member_id,signup_date,churn,outreach,url,title,description,timestamp
0,1,2024-04-12,0,0,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:00
1,1,2024-04-12,0,0,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:00
2,1,2024-04-12,0,0,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:00
3,2,2025-03-11,0,0,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:00
4,2,2025-03-11,0,0,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:00


In [ ]:
merged.columns

Index(['member_id', 'signup_date', 'churn', 'outreach', 'url', 'title',
       'description', 'timestamp'],
      dtype='object')

## claims

In [ ]:
# Load and show claims from data/train
claim_path = repo_root / "data" / "train" / "claims.csv"

claims = pd.read_csv(claim_path, low_memory=False)

# try to parse a common timestamp column if present
if "diagnosis_date" in claims.columns:
    claims["diagnosis_date"] = pd.to_datetime(
        claims["diagnosis_date"], errors="coerce", dayfirst=False
    )
claims["diagnosis_days_from_outreach"] = (outreach_time - claims["diagnosis_date"]).dt.days
print("shape:", claims.shape)
print("columns:", list(claims.columns))
claims.head(10)

shape: (64576, 4)
columns: ['member_id', 'icd_code', 'diagnosis_date', 'diagnosis_days_from_outreach']


,member_id,icd_code,diagnosis_date,diagnosis_days_from_outreach
0,1,Z71.3,2025-07-09,6
1,1,Z71.3,2025-07-14,1
2,1,J00,2025-07-12,3
3,1,M54.5,2025-07-01,14
4,1,I10,2025-07-12,3
5,1,J00,2025-07-01,14
6,1,J00,2025-07-07,8
7,1,J00,2025-07-13,2
8,1,E11.9,2025-07-09,6
9,2,E11.9,2025-07-08,7


In [ ]:
claims_count = claims.groupby("member_id").size().rename("claims_count").to_frame()

# add last claim date and days from last claim (relative to treat_time)
last_claim = claims.groupby("member_id")["diagnosis_date"].max().rename("last_claim_date")
# days from last claim (relative to outreach_time)
days_from_last = (outreach_time - last_claim).dt.days.rename("days_from_last_claim")

# count repeated claims (more than 1 of the same). Prefer icd_code, fallback to diagnosis_date.
if "icd_code" in claims.columns:
    grouped = claims.groupby(["member_id", "icd_code"]).size()
else:
    grouped = claims.groupby(["member_id", "diagnosis_date"]).size()

# number of diagnosis types per member that appear more than once
num_repeated_types = grouped[grouped > 1].groupby(level=0).size().rename("num_repeated_claim_types")

# total repeated claims beyond the first (sum of (count-1) for each duplicated diagnosis per member)
repeated_claims_count = grouped.groupby(level=0).apply(lambda s: (s - 1).clip(lower=0).sum()).rename("repeated_claims_count")

# align indices with last_claim and fill missing with 0
num_repeated_types = num_repeated_types.reindex(last_claim.index).fillna(0).astype(int)
repeated_claims_count = repeated_claims_count.reindex(last_claim.index).fillna(0).astype(int)

# combine into last_claim (so the existing join that follows will bring these cols into claims_count)
last_claim = pd.concat([last_claim, num_repeated_types, repeated_claims_count], axis=1)
claims_count = claims_count.join(last_claim).join(days_from_last)
codes = ["E11.9", "I10", "Z71.3"]
for code in codes:
    col = f"has_{code.replace('.', '_')}"
    flag = claims["icd_code"].fillna("").str.startswith(code)
    flag_series = claims.loc[flag].groupby("member_id").size().rename(col)
    flag_series = (flag_series >= 1).astype(int)
    claims_count = claims_count.join(flag_series)

# count_same_claim: max repeats of the same diagnosis per member (prefer icd_code if available)
if "icd_code" in claims.columns:
    count_same = (
        claims.groupby(["member_id", "icd_code"]).size().groupby("member_id").max().rename("count_same_claim")
    )
else:
    # fallback to diagnosis_date duplicates if no icd_code column
    count_same = (
        claims.groupby(["member_id", "diagnosis_date"]).size().groupby("member_id").max().rename("count_same_claim")
    )
claims_count.drop(columns=["last_claim_date"], inplace=True)
claims_count.head()

,claims_count,num_repeated_claim_types,repeated_claims_count,days_from_last_claim,has_E11_9,has_I10,has_Z71_3
member_id,,,,,,,
1,9,2,4,1,1.0,1.0,1.0
2,9,3,4,2,1.0,1.0,1.0
3,4,1,1,1,1.0,NaN,NaN
4,5,1,2,2,NaN,1.0,NaN
5,7,2,2,1,1.0,1.0,1.0


In [ ]:
merged = pd.merge(merged, claims_count, on="member_id", how="left")
merged.head()

,member_id,signup_date,churn,outreach,url,title,description,timestamp,claims_count,num_repeated_claim_types,repeated_claims_count,days_from_last_claim,has_E11_9,has_I10,has_Z71_3
0,1,2024-04-12,0,0,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0
1,1,2024-04-12,0,0,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0
2,1,2024-04-12,0,0,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0
3,2,2025-03-11,0,0,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0
4,2,2025-03-11,0,0,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0


## churn lables

In [ ]:
merged["days_as_user"]= (outreach_time - pd.to_datetime(merged["signup_date"], errors="coerce", dayfirst=False)).dt.days
merged.drop(columns=["signup_date"], inplace=True)
merged.head()

,member_id,churn,outreach,url,title,description,timestamp,claims_count,num_repeated_claim_types,repeated_claims_count,days_from_last_claim,has_E11_9,has_I10,has_Z71_3,days_as_user
0,1,0,0,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459
1,1,0,0,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459
2,1,0,0,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459
3,2,0,0,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0,126
4,2,0,0,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0,126


## app usage

In [ ]:
# Load and show claims from data/train
app_path = repo_root / "data" / "train" / "app_usage.csv"
app_usage = pd.read_csv(app_path, low_memory=False)

In [ ]:
app_usage.drop(columns=["event_type"], inplace=True)

In [ ]:
app_usage

,member_id,timestamp
0,1,2025-07-13 08:43:37
1,1,2025-07-14 15:28:58
2,1,2025-07-02 19:43:17
3,1,2025-07-09 09:40:14
4,1,2025-07-07 00:39:54
...,...,...
97784,10000,2025-07-01 11:05:24
97785,10000,2025-07-09 08:43:18
97786,10000,2025-07-05 11:25:41
97787,10000,2025-07-04 19:08:59


In [ ]:
app_usage["last_use"] = pd.to_datetime(app_usage["timestamp"], errors="coerce", dayfirst=True)
last_use = app_usage.groupby("member_id")["last_use"].max().rename("last_use_date")
days_from_last_use = (outreach_time - last_use).dt.days.rename("days_from_last_use")
app_usage_count = app_usage.groupby("member_id").size().rename("app_usage_count").to_frame()
app_usage_count = app_usage_count.join(days_from_last_use)
app_usage_count

C:\Users\elad\AppData\Local\Temp\ipykernel_19716\2173224832.py:1: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  app_usage["last_use"] = pd.to_datetime(app_usage["timestamp"], errors="coerce", dayfirst=True)


,app_usage_count,days_from_last_use
member_id,,
1,6,0
2,10,0
3,9,2
4,10,1
5,10,1
...,...,...
9996,8,0
9997,11,1
9998,12,3


In [ ]:
merged = pd.merge(merged, app_usage_count, on="member_id", how="left")
merged.head()

,member_id,churn,outreach,url,title,description,timestamp,claims_count,num_repeated_claim_types,repeated_claims_count,days_from_last_claim,has_E11_9,has_I10,has_Z71_3,days_as_user,app_usage_count,days_from_last_use
0,1,0,0,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459,6.0,0.0
1,1,0,0,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459,6.0,0.0
2,1,0,0,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:00,9.0,2.0,4.0,1.0,1.0,1.0,1.0,459,6.0,0.0
3,2,0,0,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0,126,10.0,0.0
4,2,0,0,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:00,9.0,3.0,4.0,2.0,1.0,1.0,1.0,126,10.0,0.0


In [ ]:
print("G")

G


In [ ]:
from src.process_datasets import *

In [ ]:
from typing import Optional
from pyparsing import Union
def build_merged_features(
    web_visits_csv: Union[str, Path],
    claims_csv: Union[str, Path],
    app_usage_csv: Union[str, Path],
    churn_csv: Union[str, Path],
    outreach_time: Optional[Union[str, datetime]] = None,
    dayfirst_web: bool = True,
    dayfirst_app: bool = True,
) -> pd.DataFrame:
    """
    Build merged features from web_visits, claims, app_usage and churn label CSVs.
    Implements the transformations present in `notebooks/new_data_process.ipynb`.

    Parameters
    ----------
    web_visits_csv, claims_csv, app_usage_csv, churn_csv:
        Paths to the CSV files.
    outreach_time:
        datetime or str. If provided will be used to compute 'days_from_outreach' features.
        If None the function will infer outreach_time as the max available timestamp across
        web_visits.timestamp, claims.diagnosis_date, app_usage.timestamp (whichever exists).
    dayfirst_web/dayfirst_app:
        Controls dayfirst parsing for those CSVs (keeps notebook behavior).

    Returns
    -------
    merged : pd.DataFrame
        The churn table merged with all derived features.
    """
    # Read files
    web_visits = pd.read_csv(web_visits_csv, low_memory=False)
    claims = pd.read_csv(claims_csv, low_memory=False)
    app_usage = pd.read_csv(app_usage_csv, low_memory=False)
    churn = pd.read_csv(churn_csv, low_memory=False)

    # Parse timestamps where present
    if "timestamp" in web_visits.columns:
        web_visits["timestamp"] = pd.to_datetime(
            web_visits["timestamp"], errors="coerce", dayfirst=dayfirst_web
        )
    if "diagnosis_date" in claims.columns:
        claims["diagnosis_date"] = pd.to_datetime(
            claims["diagnosis_date"], errors="coerce", dayfirst=False
        )
    if "timestamp" in app_usage.columns:
        app_usage["timestamp"] = pd.to_datetime(
            app_usage["timestamp"], errors="coerce",
        )
    if "signup_date" in churn.columns:
        # leave parsing until we compute days_as_user (to handle missing signup_date)
        pass

    # Infer outreach_time if not provided
    if outreach_time is None:
        candidates = []
        if "timestamp" in web_visits.columns:
            candidates.append(web_visits["timestamp"].max())
        if "diagnosis_date" in claims.columns:
            candidates.append(claims["diagnosis_date"].max())
        if "timestamp" in app_usage.columns:
            candidates.append(app_usage["timestamp"].max())
        # pick the latest non-na datetime
        candidates = [c for c in candidates if pd.notna(c)]
        if candidates:
            outreach_time = max(candidates)
        else:
            outreach_time = pd.Timestamp.now()
    else:
        if isinstance(outreach_time, str):
            outreach_time = pd.to_datetime(outreach_time, errors="coerce")
        outreach_time = pd.Timestamp(outreach_time)

    # --- WEB VISITS FEATURES ---
    # safe extraction of url_category and domain

    web_visits_for_merge = process_datasets_web_visits(web_visits, outreach_time)
    web_visits_for_merge = web_visits_for_merge.fillna(0)
    # convert to ints where safe (counts/days)
    for c in web_visits_for_merge.columns:
        # if column is numeric-like, convert to int
        if pd.api.types.is_numeric_dtype(web_visits_for_merge[c]):
            # safe cast: round then astype int
            web_visits_for_merge[c] = (
                web_visits_for_merge[c].astype(float).fillna(0)
            )
    web_visits_for_merge = web_visits_for_merge.reset_index()

    # --- CLAIMS FEATURES ---
    claims_count = claims.groupby("member_id").size().rename("claims_count").to_frame()
    if "diagnosis_date" in claims.columns:
        last_claim = (
            claims.groupby("member_id")["diagnosis_date"]
            .max()
            .rename("last_claim_date")
        )
        days_from_last = (outreach_time - last_claim).dt.days.rename(
            "days_from_last_claim"
        )
    else:
        last_claim = pd.Series(dtype="datetime64[ns]", name="last_claim_date")
        days_from_last = pd.Series(dtype="float64", name="days_from_last_claim")

    # grouped by icd_code if available else by diagnosis_date
    if "icd_code" in claims.columns:
        grouped = claims.groupby(["member_id", "icd_code"]).size()
    elif "diagnosis_date" in claims.columns:
        grouped = claims.groupby(["member_id", "diagnosis_date"]).size()
    else:
        grouped = pd.Series(dtype="int")

    if not grouped.empty:
        num_repeated_types = (
            grouped[grouped > 1]
            .groupby(level=0)
            .size()
            .rename("num_repeated_claim_types")
        )
        repeated_claims_count = (
            grouped.groupby(level=0)
            .apply(lambda s: (s - 1).clip(lower=0).sum())
            .rename("repeated_claims_count")
        )
    else:
        num_repeated_types = pd.Series(dtype="int", name="num_repeated_claim_types")
        repeated_claims_count = pd.Series(dtype="int", name="repeated_claims_count")

    # align indices and fill
    if not last_claim.empty:
        num_repeated_types = (
            num_repeated_types.reindex(last_claim.index).fillna(0).astype(int)
        )
        repeated_claims_count = (
            repeated_claims_count.reindex(last_claim.index).fillna(0).astype(int)
        )
        last_claim_df = pd.concat(
            [last_claim, num_repeated_types, repeated_claims_count], axis=1
        )
    else:
        # use index of claims_count for alignment
        last_claim_df = pd.concat(
            [
                num_repeated_types.reindex(claims_count.index).fillna(0).astype(int),
                repeated_claims_count.reindex(claims_count.index).fillna(0).astype(int),
            ],
            axis=1,
        )
        last_claim_df.index.name = "member_id"

    claims_count = claims_count.join(last_claim_df, how="left")
    # join days_from_last if exists
    if isinstance(days_from_last, pd.Series) and not days_from_last.empty:
        claims_count = claims_count.join(days_from_last, how="left")

    # ICD flag columns for codes of interest
    codes = ["E11.9", "I10", "Z71.3"]
    for code in codes:
        col = f"has_{code.replace('.', '_')}"
        if "icd_code" in claims.columns:
            flag = claims["icd_code"].fillna("").astype(str).str.startswith(code)
            flag_series = claims.loc[flag].groupby("member_id").size().rename(col)
            flag_series = (flag_series >= 1).astype(int)
            claims_count = claims_count.join(flag_series, how="left")
        else:
            # no icd_code column -> create zeros
            claims_count[col] = 0

    # count_same_claim: maximum repeats of same diagnosis per member
    if "icd_code" in claims.columns:
        count_same = (
            claims.groupby(["member_id", "icd_code"])
            .size()
            .groupby("member_id")
            .max()
            .rename("count_same_claim")
        )
    elif "diagnosis_date" in claims.columns:
        count_same = (
            claims.groupby(["member_id", "diagnosis_date"])
            .size()
            .groupby("member_id")
            .max()
            .rename("count_same_claim")
        )
    else:
        count_same = pd.Series(dtype="int", name="count_same_claim")

    claims_count = claims_count.join(count_same, how="left")

    # drop last_claim_date column as in notebook
    if "last_claim_date" in claims_count.columns:
        claims_count = claims_count.drop(columns=["last_claim_date"])

    # fill numeric NaNs with 0 and convert to int where appropriate
    for c in claims_count.columns:
        if (
            pd.api.types.is_numeric_dtype(claims_count[c])
            or claims_count[c].dtype == "float64"
        ):
            claims_count[c] = claims_count[c].fillna(0).astype(int)
        else:
            claims_count[c] = claims_count[c].fillna("")

    claims_count = claims_count.reset_index()

    # --- APP USAGE FEATURES ---
    if "event_type" in app_usage.columns:
        app_usage = app_usage.drop(columns=["event_type"])
    if "timestamp" in app_usage.columns:
        app_usage["last_use"] = app_usage["timestamp"]
        last_use = (
            app_usage.groupby("member_id")["last_use"].max().rename("last_use_date")
        )
        days_from_last_use = (outreach_time - last_use).dt.days.rename(
            "days_from_last_use"
        )
        days_from_last_use = (outreach_time - last_use).dt.days.rename("days_from_last_use")

    else:
        last_use = pd.Series(dtype="datetime64[ns]", name="last_use_date")
        days_from_last_use = pd.Series(dtype="float64", name="days_from_last_use")

    app_usage_count = (
        app_usage.groupby("member_id").size().rename("app_usage_count").to_frame()
    )
    if isinstance(days_from_last_use, pd.Series) and not days_from_last_use.empty:
        app_usage_count = app_usage_count.join(days_from_last_use, how="left")

    # fill numeric NaNs with 0
    for c in app_usage_count.columns:
        if (
            pd.api.types.is_numeric_dtype(app_usage_count[c])
            or app_usage_count[c].dtype == "float64"
        ):
            app_usage_count[c] = app_usage_count[c].fillna(0).astype(int)
        else:
            app_usage_count[c] = app_usage_count[c].fillna("")

    app_usage_count = app_usage_count.reset_index()

    # --- MERGE ALL INTO churn ---
    merged = pd.merge(churn, web_visits_for_merge, on="member_id", how="left")
    merged = pd.merge(merged, claims_count, on="member_id", how="left")
    merged = pd.merge(merged, app_usage_count, on="member_id", how="left")

    # days_as_user from signup_date if available
    if "signup_date" in merged.columns:
        merged["days_as_user"] = (
            outreach_time - pd.to_datetime(merged["signup_date"], errors="coerce")
        ).dt.days
        # drop signup_date as notebook did
        merged = merged.drop(columns=["signup_date"])

    # fill numeric feature NaNs with 0, non-numeric with empty string
    feat_cols = merged.columns.difference(churn.columns)
    for c in feat_cols:
        if pd.api.types.is_numeric_dtype(merged[c]) or merged[c].dtype == "float64":
            merged[c] = merged[c].fillna(0)
        else:
            merged[c] = merged[c].fillna("")
    treat_col = ["outreach"]
    # # --- INTERACTION FEATURES WITH OUTREACH (TREATMENT) ---
    # # Autodetect treatment column
    # possible_treat_cols = ["outreach"]
    # treat_col = None

    # for c in possible_treat_cols:
    #     if c in merged.columns:
    #         treat_col = c
    #         break

    # if treat_col is None:
    #     raise ValueError(
    #         "No treatment column found. Expected one of: treatment, outreach, outreach_flag, treat."
    #     )

    # print(f"[INFO] Building interaction features with treatment column: {treat_col}")

    # # Add numeric × treatment interaction features
    # numeric_cols = [
    #     col
    #     for col in merged.columns
    #     if col not in ["member_id", "churn", treat_col]
    #     and pd.api.types.is_numeric_dtype(merged[col])
    # ]

    # for col in numeric_cols:
    #     inter_col = f"{col}_x_{treat_col}"
    #     merged[inter_col] = merged[col] * merged[treat_col]

    # print(f"[INFO] Added {len(numeric_cols)} interaction features.")

    # --- BEHAVIORAL CHANGE FEATURES ---
    print("[INFO] Building behavioral change features...")

    # Example: for numeric usage features, compute short-term vs long-term trends
    # Here we consider columns ending with '_count', '_visits', '_usage' as candidates
    behavioral_cols = [
        col
        for col in merged.columns
        if any(suffix in col for suffix in ["_count", "_visits", "_usage"])
        and col not in ["member_id", "churn", treat_col]
    ]

    for col in behavioral_cols:
        # compute simple difference from previous period if previous period exists
        prev_col = f"{col}_prev"
        # if a previous period column exists, compute delta
        if prev_col in merged.columns:
            delta_col = f"{col}_delta"
            merged[delta_col] = merged[col] - merged[prev_col]
        else:
            # otherwise, create difference from median or mean
            trend_col = f"{col}_trend"
            median_val = merged[col].median()
            merged[trend_col] = merged[col] - median_val

    print(f"[INFO] Added {len(behavioral_cols)} behavioral change features.")

    # --- ENGAGEMENT SEGMENTS (CATEGORICAL FEATURES) ---
    print("[INFO] Building engagement segments...")

    # Numeric features to segment
    segment_cols = [
        col
        for col in merged.columns
        if any(
            suffix in col
            for suffix in ["_count", "_visits", "_usage", "_trend", "_delta"]
        )
        and col not in ["member_id", "churn", treat_col]
    ]

    n_bins = 4  # number of segments (quartiles)

    for col in segment_cols:
        seg_col = f"{col}_segment"
        try:
            # Create quartile segments, label 0=lowest, 3=highest
            merged[seg_col] = pd.qcut(
                merged[col].rank(method="first"), n_bins, labels=False
            )
        except ValueError:
            # If all values are identical, fallback to single segment
            merged[seg_col] = 0

    print(f"[INFO] Added {len(segment_cols)} engagement segment features.")

    return merged

In [558]:
merged_df_train = build_merged_features(
    repo_root / "data" / "train" / "web_visits.csv",
    repo_root / "data" / "train" / "claims.csv",
    repo_root / "data" / "train" / "app_usage.csv",
    repo_root / "data" / "train" / "churn_labels.csv",
    outreach_time="2025-07-15",  # or omit to let the function infer
    dayfirst_web=True,
)
merged_df_train.head()
merged_df_train.to_csv(
    repo_root / "data" / "train" / "churn_labels_with_all_feats.csv", index=False
)
merged_df_test = build_merged_features(
    repo_root / "data" / "test" / "test_web_visits.csv",
    repo_root / "data" / "test" / "test_claims.csv",
    repo_root / "data" / "test" / "test_app_usage.csv",
    repo_root / "data" / "test" / "test_churn_labels.csv",
    dayfirst_web=False,
    outreach_time="2025-07-15",  # or omit to let the function infer
)
merged_df_test.head()
merged_df_test.to_csv(
    repo_root / "data" / "test" / "test_churn_labels_with_all_feats.csv", index=False
)

[INFO] Building behavioral change features...
[INFO] Added 8 behavioral change features.
[INFO] Building engagement segments...
[INFO] Added 16 engagement segment features.
[INFO] Building behavioral change features...
[INFO] Added 8 behavioral change features.
[INFO] Building engagement segments...
[INFO] Added 16 engagement segment features.


In [541]:
merged_df_test[["welco_domain_ratio","member_id"]]
merged_df_train[["welco_domain_ratio", "member_id"]]

,welco_domain_ratio,member_id
0,0.666667,1
1,0.222222,2
2,0.115385,3
3,0.066667,4
4,0.000000,5
...,...,...
9995,0.222222,9996
9996,0.250000,9997
9997,0.125000,9998
9998,0.083333,9999


In [ ]:
merged_df_train["domain_care.portal_days_from_outreach"]

0       0
1       7
2       4
3       5
4       7
       ..
9995    5
9996    0
9997    0
9998    2
9999    7
Name: domain_care.portal_days_from_outreach, Length: 10000, dtype: int32